In [1]:
from binance.spot import Spot
from binance.client import Client
import pandas as pd
import requests
import datetime
import numpy as np

In [2]:
%run "Binance API.ipynb".ipynb

In [3]:
binance_api_key='E90xR7UnO2oeP3lrgC50zkqUuAqyyXs4uSlAGcqIVO9n90ELrFI4gfVI2HGGxxCj'
binance_api_secret='Inls2xYl9FqPi0zlIrucfxG81drx7w6Pv6SzYFxMNoAcwDjlCKug7zUM9zB3lF9h'
Binance=BinanceAPI(binance_api_key,binance_api_secret)

In [4]:
trade_history=pd.read_excel('Trade History.xlsx')
trade_history['Date(UTC)']=pd.to_datetime(trade_history['Date(UTC)'])
trade_history['Time']=trade_history['Date(UTC)'].dt.round('T')
trade_history=trade_history.set_index('Date(UTC)')

trade_info=zip(trade_history['Market'],trade_history.index)
trade_info=dict(enumerate(trade_info))

trade_price={}
for index in trade_info:
    
    if trade_info[index][0][-4:]=='USDT':
        ticker=trade_info[index][0]
    else:
        ticker=trade_info[index][0][-3:]+'USDT'
        
    price=Binance.binance_api.klines(ticker,interval='1m',startTime=int(trade_info[index][1].round(freq='min').timestamp()-60)*1000,limit=1)
        
    trade_price[index]=(trade_info[index][1],trade_info[index][0],price[0][4])

In [5]:
price=pd.DataFrame(trade_price.values(),columns=['Time','Market','Pair Price'])
price=pd.concat([trade_history.reset_index(),price['Pair Price']],axis=1)
price['Price in USDT']=np.where(price['Market'].str[-4:]=='USDT',price['Price'],price['Price'].astype(float)*price['Pair Price'].astype(float))
price['Total in USDT']=price['Price in USDT']*price['Amount'].astype(float)

In [6]:
price['Amount'].where(price['Type']=='BUY').cumsum()

0        57.000
1        91.000
2       185.000
3           NaN
4       258.000
5      1243.000
6      1696.000
7           NaN
8      2067.000
9           NaN
10          NaN
11          NaN
12     2067.100
13     2152.700
14          NaN
15     4129.500
16          NaN
17          NaN
18     4217.500
19     6167.500
20          NaN
21          NaN
22          NaN
23     9632.700
24          NaN
25          NaN
26          NaN
27          NaN
28          NaN
29          NaN
30          NaN
31     9633.821
32          NaN
33    10844.321
34    11926.521
35          NaN
36          NaN
37          NaN
38          NaN
39          NaN
40          NaN
41          NaN
42          NaN
43          NaN
44          NaN
45          NaN
Name: Amount, dtype: float64

In [7]:
buy=price[price['Type']=='BUY'][['Market','Amount','Price in USDT']]

buy=buy.reset_index().drop(columns='index')
buy=buy.to_dict()

traded_crypto=set()
for keys in buy['Market']:
    traded_crypto.add(buy['Market'][keys])

average_price=dict()

for crypto in traded_crypto:
    
    avg_price=0
    quantities=0
    
    for key in buy['Market']:
        
        if buy['Market'][key]==crypto:
            
            price=float(buy['Price in USDT'][key])
            avg_price+=float(buy['Amount'][key])*price
            quantities+=float(buy['Amount'][key])
    avg_price=avg_price/quantities                            
    average_price[crypto]=(avg_price,quantities)
    
average_price

crypto_=set()
for key in average_price:
        
    if key[-4:]=='USDT':
        crypto=key[:-4]
    else:
        crypto=key[:-3]
    
    crypto_.add(crypto)
    
crypto_

{'BB', 'BNB', 'JASMY', 'NOT', 'OM', 'TRX'}

In [8]:
average_price_concatenated={}

for crypto in crypto_:
    
    quantities=0
    average_price_crypto=0
    
    for t in average_price:
        if t[:len(crypto)]==crypto:
            quantities+=average_price[t][1]
            average_price_crypto+=average_price[t][1]*average_price[t][0]
            
    average_price_concatenated[crypto+'USDT']=average_price_crypto/quantities

In [9]:
book_cost_recalculated=pd.DataFrame(average_price_concatenated.values(),index=average_price_concatenated.keys(),columns=['Book Cost'])
book_cost_recalculated

,Book Cost
NOTUSDT,0.019619
BBUSDT,0.398000
JASMYUSDT,0.035358
OMUSDT,1.155439
TRXUSDT,0.164422
BNBUSDT,670.860934
